# Data

> In this part, data collection, preparation, cleaning and improvement are performed.

## 1. Import Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import re

from bs4 import BeautifulSoup
import requests

from geopy.geocoders import Nominatim
from functools import partial

___
## 2. Prepping Data

First, let's process the data from the **cian_Penza.txt** that contains information from ads, in order to obtain and transform the data into a pandas dataframe. In this step, we will use the library **BeautifulSoup**.

### 2.1. Read Data

Read Data from cian_Penza.txt.

In [2]:
with open('Data/cian_Penza.txt', 'r') as file:
    list_html = file.readlines()
    html = ''.join(list_html)
    soup = BeautifulSoup(html, 'html5lib')

___
### 2.2. Find Data

The data we need is stored in tags **'article'**. Therefore, we will find them all and save them to a variable **articles**.

In [3]:
articles = soup.find_all('article')

___
### 2.3. Clearing data

#### 2.3.1. Create Function for removing side and extra lines

In [4]:
def dellete(data):
    if 'Ещё объекты' in pre_data[5]:
        del data[5]
    if ('Подробнее' in pre_data[6]) or ('owner' in pre_data[6]):
        del data[6]
    if 'Сдаем' in pre_data[7]:
        del data[7]
    if '' == data[7]:
        data[7] = data[8]
        data[8] = '0 ₽/мес.'
    if '+7 9' not in data[9]:
        del data[9]
    if ('Собственник' in data[10]) or ('Ещё объекты' in data[10]):
        del data[10]
    if 'Ещё объекты' in data[11]:
        del data[11]
    
    return data

#### 2.3.2. Create a loop

Create a loop that will iterate over each line from the **articles**, get the necessary data and write it to the variable **data**. Also, we will exclude announcements of apartments for daily rent.

In [5]:
data = []
for i in range(0, len(articles)):
    
    a = (articles[i].find_all('a'))
    pre_data = []
    for j in range(0, len(a)):
        if j in [0, 7]:
            continue
        elif a[j].string is None:
            pre_data.append('owner')
            continue
        pre_data.append(a[j].string)
    
    pre_data.append(articles[i].find_all('img')[0].get('src'))
    
    span = (articles[i].find_all('span'))
    for j in range(1, len(span)):
        if span[j].string is None:
            pre_data.append('')
            continue
        pre_data.append(span[j].string)
        
    p = (articles[i].find_all('p'))
    for j in range(0, len(p)):
        if p[j].string is None:
            pre_data.append('')
            continue
        pre_data.append(p[j].string)
    
    
    if len(pre_data) > 13:
        prepre_data = dellete(pre_data)
        pre_data = dellete(prepre_data)
    
    if len(pre_data) < 13:
        print(pre_data)
        n = input()
        pre_data[n] = ''
    if 'Посуточно' in pre_data[12]:
        continue
    else:
        data.append(pre_data)

An example of a received ad:

In [6]:
data[0]

['Пензенская область',
 'Пенза',
 'р-н Октябрьский',
 'проспект Строителей',
 '152',
 'student city',
 'https://cdn-p.cian.site/images/43/309/101/kvartira-penza-prospekt-stroiteley-1019033481-4.jpg',
 '3-комн. кв., 64 м², 4/10 этаж',
 '11 000 ₽/мес.',
 '+7 960 325-16-...',
 '2 недели назад',
 '12 фев, 11:35',
 'От года, 11 000 ₽ + 2 000 ₽ комм. платежи (без счётчиков), комиссия 50%, без залога',
 'Сдам 3-х ком. квартиру на длительный срок по пр.Строителей 152 квартира чистая, теплая, мебелированная, имеется вся бытовая техника']

___
### 2.4. Saving data 

Save the data to a DataFrame - **data_Penza**. Use regular expressions from the library **re** to highlight specific information.

In [7]:
data_Penza = pd.DataFrame(columns=['FullAdress', 'District', 'Address',
                                   'NumHouse', 'NumRooms', 'Area', 'Floor', 
                                   'RentPrice', 'CommunalPayments',
                                   'Meters', 'Commission', 'Pledge', 'Description'])
for i,flat in enumerate(data):
    try:
        District = flat[2]
        if 'проезд' in flat[3]:
            Address = flat[3][10:] + ' ' + flat[3][:6]
        else:
            Address = flat[3]
        NumHouse = flat[4]
        try:
            about_flat = flat[7].split(', ')
            try:
                if 'Студия' in about_flat[0]:
                    NumRooms = '1' + about_flat[0]
                else:
                    NumRooms = about_flat[0]
            except:
                NumRooms = about_flat[0]
            Area = about_flat[1][:-2]
            Floor = about_flat[2][:2].replace('/', '')
        except IndexError:
            NumRooms = None
            Area = None
            Floor = None
        RentPrice = flat[8][:-7].replace(' ', '')
        info = flat[12].split(', ')
        try:
            CommunalPayments = re.findall(r'\+ (\d+ \d+)', info[1])[0].replace(' ', '')
        except IndexError:
            CommunalPayments = '0'
        try:
            Meters = re.findall(r'платежи \((.+)\)', info[1])[0]
        except IndexError:
            Meters = 'не включены'
        Commission = info[2]
        Pledge = info[3]
        Description = flat[13]
        FullAdress = Address + ', ' + NumHouse
    
        data_Penza = data_Penza.append({'FullAdress': FullAdress, 'District': District, 'Address': Address,
                                       'NumHouse': NumHouse, 'NumRooms': NumRooms, 'Area': Area, 'Floor': Floor, 
                                        'RentPrice': RentPrice, 'CommunalPayments': CommunalPayments,
                                       'Meters': Meters, 'Commission': Commission, 'Pledge': Pledge, 
                                        'Description': Description}, ignore_index=True)
    except IndexError:
        print(i)

Check the resulting df:

In [8]:
data_Penza.head()

,FullAdress,District,Address,NumHouse,NumRooms,Area,Floor,RentPrice,CommunalPayments,Meters,Commission,Pledge,Description
0,"проспект Строителей, 152",р-н Октябрьский,проспект Строителей,152,3-комн. кв.,64,4,11000,2000,без счётчиков,комиссия 50%,без залога,Сдам 3-х ком. квартиру на длительный срок по п...
1,"улица Ленина, 12",р-н Октябрьский,улица Ленина,12,3-комн. кв.,70,1,10000,3000,без счётчиков,комиссия 50%,без залога,"Большая 3х комнатная квартира 70м2,в шикарном ..."
2,"Рахманинова проезд, 5",р-н Октябрьский,Рахманинова проезд,5,1-комн. кв.,30,3,11000,2000,без счётчиков,комиссия 50%,без залога,Сдается современная 1 ком квартира в развитом ...
3,"улица Терновского, 158Б",р-н Первомайский,улица Терновского,158Б,1-комн. кв.,40,9,11000,2000,без счётчиков,комиссия 50%,залог 3 000 ₽,11 000 + ком.плат + 600 руб Инет и Тв (подключ...
4,"улица Володарского, 75",р-н Железнодорожный,улица Володарского,75,2-комн. кв.,62,2,11000,2000,без счётчиков,комиссия 50%,без залога,Сдается 2 ком кв на Володарского 75. Сдаётся д...


- **FullAdress** - Street name and house number. We'll need it to get latitude and longitude data.
- **District** - The name of the area of the city. There are 4 of them.
- **Address** - Street name.
- **NumHouse** - House number.
- **NumRooms** - Number of rooms.
- **Area** - The area of the apartment.
- **Floor** - Floor in the house.
- **RentPrice** - The cost of renting housing per month.
- **CommunalPayments** - Communal payments. Payment for light, gas, water and other meters.
- **Meters** - Are meters included in the payment?
- **Commission** - Commission to the agent, in case of a deal.
- **Pledge** - Security deposit paid for the apartment to guarantee security. Upon leaving the apartment, the deposit is refunded.
- **Description** - Description in the ad.

In [9]:
print('DataFrame has {} lines and {} columns.'.format(data_Penza.shape[0], data_Penza.shape[1]))

DataFrame has 1411 lines and 13 columns.


___
### 2.5. Freeing data from incorrect information

At this stage, we remove the incorrect or empty information from the columns. 

#### 2.5.1. Wrong city districts

In [10]:
print('----Old Values----')
print(data_Penza.District.unique().tolist())

----Old Values----
['р-н Октябрьский', 'р-н Первомайский', 'р-н Железнодорожный', 'р-н Ленинский', 'с. Засечное', 'Ладожская улица']


We see that we have 6 unique districts, but in fact there are 4 of them in Penza. Indeed, the last two meanings do not have the "р-н" subscript at the beginning, which means a city district.

In [11]:
data_Penza = data_Penza.loc[data_Penza['District'] != 'с. Засечное']
data_Penza = data_Penza.loc[data_Penza['District'] != 'Ладожская улица']

In [12]:
print('----New Values----')
print(data_Penza.District.unique().tolist())

----New Values----
['р-н Октябрьский', 'р-н Первомайский', 'р-н Железнодорожный', 'р-н Ленинский']


#### 2.5.2. Remove missing values

As mentioned above, "True" represents a missing value, "False" means the value is present in the dataset.

In [13]:
missing_data = data_Penza.isnull()

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

FullAdress
False    1406
Name: FullAdress, dtype: int64

District
False    1406
Name: District, dtype: int64

Address
False    1406
Name: Address, dtype: int64

NumHouse
False    1406
Name: NumHouse, dtype: int64

NumRooms
False    1394
True       12
Name: NumRooms, dtype: int64

Area
False    1394
True       12
Name: Area, dtype: int64

Floor
False    1394
True       12
Name: Floor, dtype: int64

RentPrice
False    1406
Name: RentPrice, dtype: int64

CommunalPayments
False    1406
Name: CommunalPayments, dtype: int64

Meters
False    1406
Name: Meters, dtype: int64

Commission
False    1406
Name: Commission, dtype: int64

Pledge
False    1406
Name: Pledge, dtype: int64

Description
False    1406
Name: Description, dtype: int64



In [14]:
data_Penza.dropna(axis=0, inplace=True)

#### 2.5.3. Remove incorrect values in house numbers

The house number can be a number or a number with a letter. If instead the value is 'Этажи Пенза', we delete these lines, since they are not suitable for further analysis.

In [15]:
data_Penza = data_Penza.loc[data_Penza['NumHouse'] != 'Этажи Пенза']

We used the values from this column to form the values in the **FullAdress** column, therefore, having removed the incorrect values in one place, we removed them everywhere. Now we can delete the entire column, since the information in it is duplicated and does not carry any usefulness.

In [16]:
data_Penza.drop(['NumHouse'], axis=1, inplace=True)

#### 2.5.4. Remove duplicates

In particular, you should **pay attention to duplicate data**. We will also get rid of them to improve the accuracy of the analysis.

In [17]:
data_Penza = data_Penza.drop_duplicates(keep='first')

Drop indexes after removing values.

In [18]:
data_Penza.reset_index(drop=True, inplace=True)

In [19]:
print('DataFrame has {} lines and {} columns.'.format(data_Penza.shape[0], data_Penza.shape[1]))

DataFrame has 427 lines and 12 columns.


As you can see, the number of rows dropped from 1411 to 427 after the cleanup. This is almost 3 times less than it was, but now we are confident that we will get more accurate analysis results.

___
### 2.6. Improving data

At this stage, we are improving the data, making it more processable.

#### 2.6.1. Number of rooms

In [20]:
print('----Old Values----')
print(data_Penza.NumRooms.unique().tolist())

----Old Values----
['3-комн. кв.', '1-комн. кв.', '2-комн. кв.', '1Студия']


It can be seen that each copy has a number at the beginning that shows the number of rooms in the apartment. The rest of the information is useless to us.

In [21]:
data_Penza['NumRooms'] = data_Penza['NumRooms'].str[0]
data_Penza.NumRooms = data_Penza.NumRooms.astype('float')
print('----New Values----')
print(data_Penza.NumRooms.unique().tolist())

----New Values----
[3.0, 1.0, 2.0]


#### 2.6.2. Cost of utilities

The high cost of utilities is one of the main problems in Penza. This is due to the inflated tariffs of management companies and a complex heat supply calculation scheme. Gas prices in Penza are higher than in Moscow (5692 rubles / cubic meter versus 4836 rubles / cubic meter). At the same time, the real estate market in the city is experiencing stagnation. Read more on [RBC](https://realty.rbc.ru/news/5cdc5f189a794735ab002b69)


In [22]:
print('----Old Values----')
print(data_Penza.Meters.unique().tolist())

----Old Values----
['без счётчиков', 'счётчики включены', 'не включены']


If utility bills are not included in the rental price, we will consider them equal to 4000 rubles. If included, except for counters, then 2000 rubles.

In [23]:
values_meters = {'без счётчиков': 2000,'не включены': 4000,'счётчики включены': 0}
data_Penza['Meters'] = data_Penza['Meters'].map(values_meters)
print('----New Values----')
print(data_Penza.Meters.unique().tolist())

----New Values----
[2000, 0, 4000]


#### 2.6.3. Rental price

In [24]:
data_Penza.RentPrice = data_Penza.RentPrice.astype('float')

#### 2.6.4. Commission

We calculate the commission as a percentage of the rental price.

In [25]:
print('----Old Values----')
print(data_Penza.Commission.unique().tolist())

----Old Values----
['комиссия 50%', 'комиссия 40%', 'без комиссии', 'комиссия 30%', 'комиссия 67%', 'комиссия 60%']


In [26]:
data_Penza['Commission'].replace({'без комиссии': 'комиссия 0%'}, regex=True, inplace=True)
data_Penza['Commission'] = data_Penza['Commission'].str.replace('комиссия ', '').str.replace('%', '')
data_Penza['Commission'] = data_Penza['Commission'].astype('float') / 100 * data_Penza['RentPrice']

In [27]:
print('----New Values----')
print(data_Penza.Commission.unique().tolist())

----New Values----
[5500.0, 5000.0, 4000.0, 6000.0, 7500.0, 4500.0, 0.0, 4750.0, 3500.0, 7000.0, 11500.0, 6500.0, 3750.0, 3600.0, 3000.0, 3250.0, 4250.0, 3300.0, 8500.0, 6250.0, 2400.0, 2750.0, 2500.0, 9000.0, 8000.0, 10000.0, 4800.0, 3200.0, 15000.0, 6030.0, 5750.0]


#### 2.6.5. Pledge

We only need a number.

In [28]:
print('----Old Values----')
print(data_Penza.Pledge.unique().tolist())

----Old Values----
['без залога', 'залог 3 000\xa0₽', 'залог 10 000\xa0₽', 'залог 5 000\xa0₽', 'залог 7 500\xa0₽', 'залог 7 000\xa0₽', 'залог 12 500\xa0₽', 'залог 6 000\xa0₽', 'залог 2 000\xa0₽', 'залог 500\xa0₽', 'залог 2 500\xa0₽', 'залог 13 000\xa0₽', 'залог 18 000\xa0₽', 'залог 1 000\xa0₽', 'залог 12 000\xa0₽', 'залог 20 000\xa0₽', 'залог 4 000\xa0₽', 'залог 30 000\xa0₽']


In [29]:
data_Penza['Pledge'].replace({'без залога': 'залог 0 ₽'}, regex=True, inplace=True)
data_Penza['Pledge'] = data_Penza['Pledge'].str.replace('залог ', '').str.replace('₽', '').str.replace('\xa0', '').str.replace(' ', '')
data_Penza['Pledge'] = data_Penza['Pledge'].astype('float')

In [30]:
print('----New Values----')
print(data_Penza.Pledge.unique().tolist())

----New Values----
[0.0, 3000.0, 10000.0, 5000.0, 7500.0, 7000.0, 12500.0, 6000.0, 2000.0, 500.0, 2500.0, 13000.0, 18000.0, 1000.0, 12000.0, 20000.0, 4000.0, 30000.0]


#### 2.6.6. Communal Payments

In [31]:
print('----Old Values----')
print(data_Penza.CommunalPayments.unique().tolist())

----Old Values----
['2000', '3000', '1000', '3500', '0', '1500', '2500', '1200', '1700', '1800', '1100', '2200', '6000', '1300', '4000', '1600', '5000']


In [32]:
data_Penza['CommunalPayments'] = data_Penza['CommunalPayments'].astype('float')

In [33]:
print('----New Values----')
print(data_Penza.CommunalPayments.unique().tolist())

----New Values----
[2000.0, 3000.0, 1000.0, 3500.0, 0.0, 1500.0, 2500.0, 1200.0, 1700.0, 1800.0, 1100.0, 2200.0, 6000.0, 1300.0, 4000.0, 1600.0, 5000.0]


#### 2.6.7. Floor and Area

Change data type for apartment area and floors.

In [34]:
data_Penza.Area = data_Penza.Area.str.replace(",", ".")
data_Penza[['Area', 'Floor']] = data_Penza[['Area', 'Floor']].astype('float')

#### 2.6.6. Payments

Let's collect all the payment data in two separate columns. 
1. **MonthlyPayments** - payments that must be made every month in case of renting an apartment. They include **RentPrice, Meters, CommunalPayments**.
2. **OneTimeFee** - payment that must be paid once at the very beginning of renting an apartment. They include **Commission, Pledge**.

In [35]:
data_Penza['MonthlyPayments'] = data_Penza.RentPrice + data_Penza.Meters + data_Penza.CommunalPayments
data_Penza['OneTimeFee'] = data_Penza.Commission + data_Penza.Pledge

In [36]:
data_Penza.drop(['RentPrice', 'Meters', 'CommunalPayments', 'Commission', 'Pledge'], axis=1, inplace=True)

For the convenience of counting, divide the resulting columns by 1000. This is how they will express the amount in thousands of rubles.

In [37]:
data_Penza['MonthlyPayments'] = data_Penza['MonthlyPayments'] / 1000
data_Penza['OneTimeFee'] = data_Penza['OneTimeFee'] / 1000

In [38]:
data_Penza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FullAdress       427 non-null    object 
 1   District         427 non-null    object 
 2   Address          427 non-null    object 
 3   NumRooms         427 non-null    float64
 4   Area             427 non-null    float64
 5   Floor            427 non-null    float64
 6   Description      427 non-null    object 
 7   MonthlyPayments  427 non-null    float64
 8   OneTimeFee       427 non-null    float64
dtypes: float64(5), object(4)
memory usage: 30.1+ KB


____
### 2.7. Add GeoData

Geodata means **latitude** and **longitude** for each address in our Dataframe. We'll use the **geopy** library to get the data.

#### 2.7.1. Сreate a df for Location.

We will save the data in this Dataframe.

In [39]:
Location = pd.DataFrame(columns=['microDistrict', 'ALatitude', 'ALongitude', 'Latitude', 'Longitude'])

#### 2.7.2. Getting location data

Function to check for a number.

In [40]:
def is_not_number(s):
    try:
        number = int(s)
        return False
    except ValueError:
        return True

In this loop, we go through each address and district in our Dataframe. If we have not received the data, we try to send the request again. We print every unsuccessful attempt. If the request is successful, then add data to Dataframe.

In [41]:
geolocator = Nominatim(user_agent="to_explorer")
geocode = partial(geolocator.geocode, language="ru")
for i in range(len(data_Penza)):
    address = data_Penza.FullAdress[i] + ', Пенза'
    district = data_Penza.District[i] + ', Пенза'
    try:
        location = geolocator.geocode(district)
        latitude = location.latitude
        longitude = location.longitude
        alocation = geolocator.geocode(address)
        alatitude = alocation.latitude
        alongitude = alocation.longitude
        row = alocation.address.split(', ')
        if is_not_number(row[0][0]):
            if is_not_number(row[1][0]):
                row.insert(0, '')
            else:
                del row[0]
        
        if ('район' not in row[3].split()) and ('Пензенская' not in row[3].split()) and ('Пенза' not in row[3].split()):
            del row[2]
        
        if ('Ленинский район' == row[2]) or ('Первомайский район' == row[2]):
            naibourhood = 'Западная поляна'
        elif 'кордон Октябрьский Сад' == row[2]:
            naibourhood = 'Окружная'
        elif 'городское поселение Нижний Ломов' == row[2]:
            raise AttributeError
        else:
            naibourhood = row[2]
        Location = Location.append({'microDistrict': naibourhood, 'ALatitude': alatitude, 'ALongitude': alongitude, 'Latitude': latitude, 'Longitude': longitude}, ignore_index=True)
    except AttributeError:
        print(i, alocation)
        Location = Location.append({'microDistrict': np.NaN, 'ALatitude': np.NaN, 'ALongitude': np.NaN, 'Latitude': latitude, 'Longitude': longitude}, ignore_index=True)

73 None
126 None
198 None
201 None
216 None
276 None
318 None
323 None
353 None
355 None
365 None
410 3, улица Лермонтова, Нижний Ломов, городское поселение Нижний Ломов, Нижнеломовский район, Пензенская область, Приволжский федеральный округ, 442150, Россия


We were unable to receive data for 12 addresses. 

In [42]:
Location.head()

,microDistrict,ALatitude,ALongitude,Latitude,Longitude
0,Запрудный,53.220538,44.888742,53.172801,44.973169
1,Заводской,53.215607,45.000883,53.172801,44.973169
2,Арбеково,53.225680,44.929247,53.172801,44.973169
3,Терновка,53.144821,45.024579,53.162396,45.029846
4,Центр,53.200063,45.016249,53.203561,45.012268


#### 2.7.3. Concatenate Dataframes

Concatenate our two df's into one new one and remove the lines with empty data.

In [43]:
df_Penza = pd.concat([data_Penza, Location], axis=1)

#### 2.7.4. Delete lines with None values

Delete lines with empty values in latitude and longitude.

In [44]:
df_Penza.dropna(subset=['Latitude'], axis=0, inplace=True)
df_Penza.reset_index(drop=True, inplace=True)

Checking Data:

In [45]:
print('DataFrame has {} lines and {} columns.'.format(df_Penza.shape[0], df_Penza.shape[1]))

DataFrame has 427 lines and 14 columns.


In [46]:
df_Penza.head()

,FullAdress,District,Address,NumRooms,Area,Floor,Description,MonthlyPayments,OneTimeFee,microDistrict,ALatitude,ALongitude,Latitude,Longitude
0,"проспект Строителей, 152",р-н Октябрьский,проспект Строителей,3.0,64.0,4.0,Сдам 3-х ком. квартиру на длительный срок по п...,15.0,5.5,Запрудный,53.220538,44.888742,53.172801,44.973169
1,"улица Ленина, 12",р-н Октябрьский,улица Ленина,3.0,70.0,1.0,"Большая 3х комнатная квартира 70м2,в шикарном ...",15.0,5.0,Заводской,53.215607,45.000883,53.172801,44.973169
2,"Рахманинова проезд, 5",р-н Октябрьский,Рахманинова проезд,1.0,30.0,3.0,Сдается современная 1 ком квартира в развитом ...,15.0,5.5,Арбеково,53.225680,44.929247,53.172801,44.973169
3,"улица Терновского, 158Б",р-н Первомайский,улица Терновского,1.0,40.0,9.0,11 000 + ком.плат + 600 руб Инет и Тв (подключ...,15.0,8.5,Терновка,53.144821,45.024579,53.162396,45.029846
4,"улица Володарского, 75",р-н Железнодорожный,улица Володарского,2.0,62.0,2.0,Сдается 2 ком кв на Володарского 75. Сдаётся д...,15.0,5.5,Центр,53.200063,45.016249,53.203561,45.012268


In [47]:
df_Penza['microDistrict'].value_counts()

Глобус                      59
Центр                       47
Запрудный                   44
Арбеково                    41
ГПЗ                         39
Западная поляна             37
Южная поляна                37
ЖК «Арбековская застава»    20
Сосновка                    19
Новая Западная поляна       18
остров Пески                17
Стрела                      12
Заводской                    9
Северная поляна              3
Шуист                        2
Гидрострой                   2
Терновка                     2
Цыганский посёлок            2
пос.Ахуны                    1
Железнодорожный район        1
Бугровка                     1
Окружная                     1
Заречный                     1
Name: microDistrict, dtype: int64

___
### 2.8. Utilizing the Foursquare API

At this stage, we will use Foursquare API to get venues for each address in Dataframe.

The Places API offers real-time access to Foursquare’s global database of rich venue data and user content to power location-based experiences in app or website.

#### 2.8.1. Preparing to connect

To connect, we have to enter the following 4 parameters.

In [51]:
CLIENT_ID = 'LBI0N4QMTJUEELDUQPYBEEDT1V5S3IPO3J4N1R05LJIK0YHF' # Foursquare ID
CLIENT_SECRET = 'NX2AS300KTMMU21U5NZAHSDXH4CVOJGTO0FPLIE1WCL2NVUS' # Foursquare Secret
VERSION = '20210131' # Foursquare API version
LIMIT = 500 # A default Foursquare API limit value

#### 2.8.2. Receiving data

Create a function that, using the connection data, will receive all venues for each micro districts within a radius of 1000 meters.

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['microDistrict', 
                             'microDistrict Latitude', 
                             'microDistrict Longitude',
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [53]:
penza_microdistrics_venues = getNearbyVenues(names=df_Penza['microDistrict'],
                                             latitudes=df_Penza['Latitude'],
                                             longitudes=df_Penza['Longitude'],
                                             radius=1000)

Checking Data:

In [54]:
print('DataFrame with coordinates has {} lines and {} columns.'.format(penza_microdistrics_venues.shape[0], penza_microdistrics_venues.shape[1]))
penza_microdistrics_venues.head()

DataFrame with coordinates has 16955 lines and 7 columns.


,microDistrict,microDistrict Latitude,microDistrict Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Запрудный,53.172801,44.973169,Spar,53.165857,44.979529,Grocery Store
1,Запрудный,53.172801,44.973169,Мой,53.167134,44.982601,Shopping Mall
2,Запрудный,53.172801,44.973169,Моя Родня,53.168849,44.968008,Shopping Mall
3,Запрудный,53.172801,44.973169,Стрелец,53.168782,44.984803,Food & Drink Shop
4,Запрудный,53.172801,44.973169,Duty Free,53.167948,44.984070,Liquor Store


In [55]:
print('----Top 10 venues by quantity----')
print(penza_microdistrics_venues.Venue.value_counts()[:10])

----Top 10 venues by quantity----
Автосуши               376
M.Видео                376
Duty Free              321
Перекресток            268
Спорт-бар "Колизей"    188
Fellini                188
Пассаж                 188
Rush                   188
Спортмастер            188
Чарли Gold             188
Name: Venue, dtype: int64


#### 2.8.3. One hot encoding

One hot Encoding is a type of vector representation in which all of the elements in a vector are 0, except for one, which has 1 as its value, where 1 represents a boolean specifying a category of the element.

In [56]:
penza_onehot = pd.get_dummies(penza_microdistrics_venues[['Venue Category']], prefix="", prefix_sep="")

Add neighborhood column back to dataframe.

In [57]:
penza_onehot['microDistrict'] = penza_microdistrics_venues['microDistrict'] 

Move neighborhood column to the first column.

In [58]:
fixed_columns = [penza_onehot.columns[-1]] + list(penza_onehot.columns[:-1])
penza_onehot = penza_onehot[fixed_columns]

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [59]:
penza_grouped_by_microDistrict = penza_onehot.groupby('microDistrict').mean().reset_index()

Checking new Dataframe:

In [60]:
print('New DataFrame with sorted venues has {} lines and {} columns.'.format(penza_grouped_by_microDistrict.shape[0], penza_grouped_by_microDistrict.shape[1]))

New DataFrame with sorted venues has 23 lines and 62 columns.


In [61]:
penza_grouped_by_microDistrict.head()

,microDistrict,ATM,American Restaurant,Asian Restaurant,BBQ Joint,Bar,Bookstore,Brewery,Burger Joint,Café,Camera Store,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Dessert Shop,Donut Shop,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Food & Drink Shop,French Restaurant,Gaming Cafe,German Restaurant,Grocery Store,Gym / Fitness Center,Hookah Bar,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Jewelry Store,Karaoke Bar,Liquor Store,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Nightclub,Park,Pharmacy,Pizza Place,Plaza,Pool,Pub,Restaurant,Shoe Store,Shopping Mall,Skating Rink,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Theater,Toy / Game Store,Wine Bar,Wine Shop
0,Арбеково,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Бугровка,0.010000,0.010000,0.010000,0.010000,0.040000,0.020000,0.010000,0.010000,0.070000,0.010000,0.010000,0.020000,0.030000,0.0,0.010000,0.010000,0.010000,0.000000,0.010000,0.030000,0.020000,0.020000,0.010000,0.010000,0.010000,0.010000,0.010000,0.020000,0.020000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.020000,0.010000,0.010000,0.010000,0.020000,0.010000,0.020000,0.010000,0.020000,0.030000,0.010000,0.020000,0.070000,0.010000,0.060000,0.010000,0.020000,0.030000,0.010000,0.010000,0.000000,0.040000,0.030000,0.000000,0.000000,0.010000
2,ГПЗ,0.000000,0.016129,0.000000,0.000000,0.032258,0.016129,0.000000,0.000000,0.080645,0.016129,0.000000,0.000000,0.016129,0.0,0.016129,0.016129,0.016129,0.016129,0.016129,0.048387,0.016129,0.032258,0.016129,0.016129,0.016129,0.032258,0.016129,0.000000,0.000000,0.016129,0.016129,0.016129,0.000000,0.016129,0.016129,0.000000,0.000000,0.000000,0.000000,0.016129,0.048387,0.016129,0.016129,0.016129,0.016129,0.000000,0.016129,0.080645,0.000000,0.032258,0.000000,0.000000,0.032258,0.016129,0.000000,0.032258,0.064516,0.032258,0.016129,0.016129,0.000000
3,Гидрострой,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Глобус,0.009911,0.009911,0.009911,0.009911,0.039643,0.019822,0.009911,0.009911,0.069376,0.009911,0.009911,0.019822,0.029732,0.0,0.009911,0.009911,0.009911,0.000000,0.009911,0.029732,0.019822,0.021606,0.009911,0.009911,0.009911,0.011695,0.009911,0.019822,0.019822,0.009911,0.009911,0.000000,0.009911,0.009911,0.011695,0.019822,0.009911,0.009911,0.009911,0.019822,0.009911,0.019822,0.009911,0.019822,0.029732,0.009911,0.019822,0.069376,0.009911,0.063033,0.009911,0.019822,0.029732,0.009911,0.009911,0.000000,0.039643,0.029732,0.000000,0.000000,0.009911


#### 2.8.4. Sort the venues

Function to sort the venues in descending order.

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create columns according to number of 10 top venues.

In [63]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['microDistrict']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

Create a new Dataframe.

In [64]:
neighborhoods_venues_sorted_microDistrict = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_microDistrict['microDistrict'] = penza_grouped_by_microDistrict['microDistrict']

for ind in np.arange(penza_grouped_by_microDistrict.shape[0]):
    neighborhoods_venues_sorted_microDistrict.iloc[ind, 1:] = return_most_common_venues(penza_grouped_by_microDistrict.iloc[ind, :], num_top_venues)

Checking new Dataframe:

In [65]:
print('New DataFrame with sorted venues has {} lines and {} columns.'.format(neighborhoods_venues_sorted_microDistrict.shape[0], neighborhoods_venues_sorted_microDistrict.shape[1]))

New DataFrame with sorted venues has 23 lines and 11 columns.


In [66]:
neighborhoods_venues_sorted_microDistrict

,microDistrict,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Арбеково,Shopping Mall,Grocery Store,Liquor Store,Food & Drink Shop,Wine Shop,Cosmetics Shop,Gym / Fitness Center,German Restaurant,Gaming Cafe,French Restaurant
1,Бугровка,Café,Restaurant,Shopping Mall,Sushi Restaurant,Bar,Electronics Store,Coffee Shop,Sporting Goods Shop,Plaza,Theater
2,ГПЗ,Restaurant,Café,Sushi Restaurant,Electronics Store,Nightclub,Food & Drink Shop,Bar,Shopping Mall,Sporting Goods Shop,Supermarket
3,Гидрострой,Convenience Store,Restaurant,Food & Drink Shop,Eastern European Restaurant,Hotel,Bar,Bookstore,Hookah Bar,Gym / Fitness Center,Grocery Store
4,Глобус,Café,Restaurant,Shopping Mall,Sushi Restaurant,Bar,Electronics Store,Theater,Coffee Shop,Plaza,Sporting Goods Shop
5,ЖК «Арбековская застава»,Shopping Mall,Grocery Store,Liquor Store,Food & Drink Shop,Wine Shop,Cosmetics Shop,Gym / Fitness Center,German Restaurant,Gaming Cafe,French Restaurant
6,Железнодорожный район,Restaurant,Café,Sushi Restaurant,Electronics Store,Nightclub,Food & Drink Shop,Bar,Shopping Mall,Sporting Goods Shop,Supermarket
7,Заводской,Shopping Mall,Grocery Store,Liquor Store,Food & Drink Shop,Wine Shop,Cosmetics Shop,Gym / Fitness Center,German Restaurant,Gaming Cafe,French Restaurant
8,Западная поляна,Convenience Store,Restaurant,Food & Drink Shop,Eastern European Restaurant,Hotel,Bar,Bookstore,Hookah Bar,Gym / Fitness Center,Grocery Store
9,Запрудный,Shopping Mall,Grocery Store,Liquor Store,Food & Drink Shop,Wine Shop,Cosmetics Shop,Gym / Fitness Center,German Restaurant,Gaming Cafe,French Restaurant


___
### 2.9. Saving the resulting tables

In [67]:
df_Penza.to_csv('df_Penza.csv', encoding='utf-8', index=False)
penza_grouped_by_microDistrict.to_csv('penza_grouped_by_microDistrict.csv', encoding='utf-8', index=False)
neighborhoods_venues_sorted_microDistrict.to_csv('neighborhoods_venues_sorted_microDistrict.csv', encoding='utf-8', index=False)